# Импорт необходимых библиотек

In [1]:
!pip install --proxy 10.0.0.1:3128 gensim emoji pandas nltk pymorphy2

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np
from random import randint

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

# Датасет ВК

## Загрузка датасета

In [7]:
colnames = ['comment',]

In [8]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [9]:
comments

,comment
0,['😭']
1,"['хранить', 'господь', 'неравнодушный', 'челов..."
2,"['евгений', 'еслиб', 'бог', 'такой', 'допускать']"
3,"['кирилл', 'полностью', 'согласный', '😢']"
4,"['кирилл', 'бог', 'путь', 'неисповедимый']"
...,...
50779,"['с', 'фамилия', '🙄']"
50780,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
50781,"['чудо', 'чудной', 'диво', 'дивный']"
50782,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [10]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [11]:
comments['comment'] = comments['comment'].apply(str_to_list)

### Выделение эмодзи флагов

In [12]:
def split(s):
    return[char for char in s]

reg_ident = ['🇷', '🇺', '🇦', '🇸', '🇧', '🇨', '🇩', '🇪', '🇫', '🇬', '🇭', '🇮', '🇯', '🇰', '🇱', '🇲', '🇳', '🇴', '🇵', '🇶', '🇸', '🇹', 'v', '🇼', '🇽', '🇾', '🇿']

def get_flags(doc):
    for item in doc:
        new_item = []
        for c in split(item):
            if c in reg_ident:
                if len(new_item) < 2:
                    new_item.append(c)
        if len(new_item) == 2 and new_item[0] != new_item[1]:
            try:
                i = doc.index(item)
                doc[i] = new_item[0] + new_item[1]
            except ValueError:
                doc[i] = ''
    return doc

In [13]:
comments['comment'] = comments['comment'].apply(get_flags)

In [14]:
comments

,comment
0,[😭]
1,"[хранить, господь, неравнодушный, человек, 🙏, 🕊]"
2,"[евгений, еслиб, бог, такой, допускать]"
3,"[кирилл, полностью, согласный, 😢]"
4,"[кирилл, бог, путь, неисповедимый]"
...,...
50779,"[с, фамилия, 🙄]"
50780,"[наш, день, повезти, мужик, фамилия, но, суть,..."
50781,"[чудо, чудной, диво, дивный]"
50782,"[пригреться, уснуть, свариться, лягушка, так, ..."


In [15]:
true_vk_data = comments['comment'].values.tolist()

# Модель Word2Vec

In [16]:
model_vk_emoji = wv(vector_size=300, min_count=3, window=5)

In [17]:
model_vk_emoji.build_vocab(true_vk_data)

In [18]:
len(model_vk_emoji.wv)

13687

In [19]:
model_vk_emoji.train(true_vk_data, total_examples=model_vk_emoji.corpus_count, epochs=1)

(349324, 394259)

In [20]:
model_vk_emoji.wv.similarity('россия', '🇷🇺')

0.9948811

In [26]:
model_vk_emoji.wv.most_similar('😠')

[('виктория', 0.9023991227149963),
 ('бегать', 0.9014540314674377),
 ('😰', 0.9013167023658752),
 ('нынешний', 0.901007354259491),
 ('вооружение', 0.900209903717041),
 ('медицина', 0.8998721837997437),
 ('приходить', 0.8997824192047119),
 ('приводить', 0.8997141122817993),
 ('плевать', 0.8997119665145874),
 ('тихий', 0.89943528175354)]

#### Экспорт модели 

In [27]:
model_vk_emoji.save("word2vec.wordvectors")

# Подготовка датасета для анализа тональности  *(дополнительно)* 

In [20]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️']

In [21]:
sarcastic = ['🤡', '🙃', '🤢', '😈']  # Нужно дополнить

In [22]:
positive = ['😀', '😃', '😉']  # Нужно дополнить